In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import bs4 as bs
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

f = open('op.txt','w') #opens a file to write

try:                           #Exception Handling
    html = urlopen("https://www.imdb.com/chart/top")
except HTTPError as e:
    print("The server returned a HTTP error")
except URLError as e:
    print("The server could not be found!")
else:
    soup=bs.BeautifulSoup(html,'lxml')
    print(soup.prettify(),file=f) #writes the source code to a text file
    f = open("op.txt", "r")
    print(f.read()) #reads the file and prints the source code

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
  <style>
   body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
  </style>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   IMDb Top 250 - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {w

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('imdb.jl', 'w')
        self.file1 = open('imdb.json', 'w')

    def close_spider(self, spider):
        self.file.close()
        self.file1.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        self.file1.write(line)
        return item

In [3]:
import logging

class IMDB(scrapy.Spider):
    name = "imdb"
    start_urls = ['https://www.imdb.com/chart/top']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'imdb.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for db in response.css('tbody.lister-list tr'):
            yield {
                'position': db.css('td.titleColumn::text').extract_first(),
                'name': db.css('td.titleColumn a::text').extract_first(),
                'year': db.css('td.titleColumn span.secondaryInfo::text').extract_first(),
                'artists':db.css("td.titleColumn a::attr(title)").extract_first(),
                'rating':db.css('td strong::text').extract_first(),                
                'poster':db.css("td.posterColumn img::attr(src)").extract_first(),
                'moive_link':db.css("td.posterColumn a::attr(href)").extract_first(),
            }


In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(IMDB)
process.start()

2019-02-12 18:35:46 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-02-12 18:35:46 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Linux-4.15.0-45-generic-x86_64-with-debian-buster-sid
2019-02-12 18:35:46 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'imdb.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [5]:
import pandas as pd
dfjl = pd.read_json('imdb.jl', lines=True)
dfjl

,artists,moive_link,name,position,poster,rating,year
0,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,The Shawshank Redemption,1,https://m.media-amazon.com/images/M/MV5BMDFkYT...,9.2,(1994)
1,"Francis Ford Coppola (dir.), Marlon Brando, Al...",/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,The Godfather,2,https://m.media-amazon.com/images/M/MV5BM2MyNj...,9.2,(1972)
2,"Francis Ford Coppola (dir.), Al Pacino, Robert...",/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,The Godfather: Part II,3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,9.0,(1974)
3,"Christopher Nolan (dir.), Christian Bale, Heat...",/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,The Dark Knight,4,https://m.media-amazon.com/images/M/MV5BMTMxNT...,9.0,(2008)
4,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb",/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,12 Angry Men,5,https://m.media-amazon.com/images/M/MV5BMWU4N2...,8.9,(1957)
5,"Steven Spielberg (dir.), Liam Neeson, Ralph Fi...",/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,Schindler's List,6,https://m.media-amazon.com/images/M/MV5BNDE4OT...,8.9,(1993)
6,"Peter Jackson (dir.), Elijah Wood, Viggo Morte...",/title/tt0167260/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,The Lord of the Rings: The Return of the King,7,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,8.9,(2003)
7,"Quentin Tarantino (dir.), John Travolta, Uma T...",/title/tt0110912/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,Pulp Fiction,8,https://m.media-amazon.com/images/M/MV5BNGNhMD...,8.9,(1994)
8,"Sergio Leone (dir.), Clint Eastwood, Eli Wallach",/title/tt0060196/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,"The Good, the Bad and the Ugly",9,https://m.media-amazon.com/images/M/MV5BOTQ5ND...,8.8,(1966)
9,"David Fincher (dir.), Brad Pitt, Edward Norton",/title/tt0137523/?pf_rd_m=A2FGELUUNOQJNL&pf_rd...,Fight Club,10,https://m.media-amazon.com/images/M/MV5BMjJmYT...,8.8,(1999)
